In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
# sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions
from Bio.PDB.Polypeptide import one_to_three
from Bio.PDB.Polypeptide import three_to_one
from Bio.PDB.PDBParser import PDBParser
from pyCodeLib import *
from small_script.myFunctions import *
sys.path.insert(0, "/Users/weilu/openmmawsem")
from helperFunctions.myFunctions import *
from collections import defaultdict
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['figure.figsize'] = np.array([16.18033, 10])    #golden ratio
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update({'font.size': 22})

In [4]:
def do(cmd, get=False, show=True):
    if get:
        out = subprocess.Popen(cmd,stdout=subprocess.PIPE,shell=True).communicate()[0].decode()
        if show:
            print(out, end="")
        return out
    else:
        return subprocess.Popen(cmd, shell=True).wait()

In [26]:
pre = "/Users/weilu/Research/server/mar_2020/3DROBOT_DATA/3DRobot_set/3VZ9B/"
nativePdb = f"{pre}/native.pdb"
decoy_list = glob.glob(f"{pre}/decoy*.pdb")
info_ = []
p = PDBParser(QUIET=True)

for decoyPdb in decoy_list[:3]:
    decoyName = decoyPdb.split("/")[-1].split(".")[0]
    # decoyPdb = decoy_list[0]
    s = p.get_structure("decoy", decoyPdb)
    decoy_s = s[0]
    q = float(do(f"python ~/opt/small_script/CalcQValueFromTwoPdb_2.py {nativePdb} {decoyPdb}", get=True))
    info_.append([decoyName, q, decoy_s])
data = pd.DataFrame(info_, columns=["Protein", "Qw", "structure"]).sort_values("Qw").reset_index(drop=True)

0.358 0.311 0.287 

In [28]:
data

,Protein,Qw,structure
0,decoy12_3,0.287,"(((<Atom H>, <Atom N>, <Atom CA>, <Atom C>, <A..."
1,decoy28_37,0.311,"(((<Atom H>, <Atom N>, <Atom CA>, <Atom C>, <A..."
2,decoy29_130,0.358,"(((<Atom H>, <Atom N>, <Atom CA>, <Atom C>, <A..."


In [23]:
a = pd.read_pickle("/Users/weilu/Research/server/mar_2020/mass_iterative_optimization/optimization_native/decoys/openMM/1osa_iteration_start_native.pkl")

In [24]:
a

,Steps,Qw,Rg,Backbone,Rama,Contact,Fragment,Exclude_Side,SideChain,Exclude,Beta,Pap,Helical,Total,Run,Protein,Folder,structure
0,202,0.46,19.97,210.54,-967.83,-269.01,-603.63,18.53,48.78,21.30,-46.46,-0.00,0.0,-1587.78,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
1,203,0.48,20.01,201.62,-981.04,-274.77,-612.63,26.64,48.26,23.99,-49.15,-0.00,0.0,-1617.08,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
2,204,0.51,20.38,202.44,-980.54,-283.50,-622.67,20.83,51.62,19.52,-48.10,-0.00,0.0,-1640.40,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
3,205,0.48,20.62,236.61,-970.28,-286.89,-610.83,21.13,62.85,21.41,-45.77,-0.00,0.0,-1571.78,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
4,206,0.47,20.15,218.81,-980.37,-284.72,-603.91,22.06,64.38,21.69,-50.29,-0.00,0.0,-1592.36,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
5,207,0.43,20.00,190.40,-978.31,-291.78,-597.74,20.73,47.98,20.20,-45.37,-0.00,0.0,-1633.88,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
6,208,0.50,20.34,227.69,-989.31,-278.55,-619.28,20.06,46.63,22.36,-48.09,-0.00,0.0,-1618.50,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
7,209,0.44,19.67,213.91,-971.53,-287.35,-609.09,28.55,58.41,19.92,-47.73,-0.00,0.0,-1594.90,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
8,210,0.47,19.77,199.47,-979.35,-286.19,-618.39,25.39,61.41,15.91,-46.64,-0.00,0.0,-1628.39,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
9,211,0.51,20.47,214.76,-985.16,-281.28,-609.80,25.73,58.11,14.24,-49.30,-0.00,0.0,-1612.72,0,1osa,iteration_start_native,"(((<Atom CA>, <Atom C>, <Atom O>, <Atom CB>), ..."
